In [1]:
# Ocean Data Testing and Comparison
# This notebook compares wave data from NDBC buoys via our ocean_data package against Surfline data

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import requests
from datetime import datetime, timedelta
import pytz
import json
import os
from bs4 import BeautifulSoup
import re

# Import our ocean_data package
from ocean_data import swell, meteorology, tide, location

In [2]:
# Define testing parameters
# These will be used throughout the notebook

# Test locations
test_locations = ["lido", "manasquan", "belmar", "rockaways"]

# Date range to test
eastern = pytz.timezone('America/New_York')
end_date = datetime.now(eastern)
start_date = end_date - timedelta(days=3)  # Get data for the last 3 days

# Generate test timestamps (every 6 hours)
test_timestamps = []
current = start_date
while current <= end_date:
    test_timestamps.append(current)
    current = current + timedelta(hours=6)

print(f"Testing {len(test_locations)} locations with {len(test_timestamps)} timestamps per location")
print(f"Date range: {start_date.strftime('%Y-%m-%d %H:%M')} to {end_date.strftime('%Y-%m-%d %H:%M')}")

Testing 4 locations with 13 timestamps per location
Date range: 2025-05-11 16:38 to 2025-05-14 16:38


In [3]:
# Functions to get data from our ocean_data package

def get_slapp_swell_data(location_name, timestamp):
    """Get swell data from our ocean_data package for a specific location and time"""
    try:
        # Get buoy mapping for location
        buoy_info = location.get_buoys_for_location(location_name)
        if not buoy_info:
            return None
        
        # Fetch swell data
        return swell.fetch_swell_data(buoy_info["swell"], timestamp)
    except Exception as e:
        print(f"Error fetching swell data: {str(e)}")
        return None

def get_slapp_met_data(location_name, timestamp):
    """Get meteorological data from our ocean_data package for a specific location and time"""
    try:
        # Get buoy mapping for location
        buoy_info = location.get_buoys_for_location(location_name)
        if not buoy_info:
            return None
        
        # Fetch meteorological data
        return meteorology.fetch_meteorological_data(buoy_info["met"], timestamp)
    except Exception as e:
        print(f"Error fetching met data: {str(e)}")
        return None

def get_slapp_tide_data(location_name, timestamp):
    """Get tide data from our ocean_data package for a specific location and time"""
    try:
        # Get buoy mapping for location
        buoy_info = location.get_buoys_for_location(location_name)
        if not buoy_info:
            return None
        
        # Fetch tide data
        return tide.fetch_water_level(buoy_info["tide"], timestamp)
    except Exception as e:
        print(f"Error fetching tide data: {str(e)}")
        return None

def get_buoy_id_for_location(location_name, data_type="swell"):
    """Get the buoy/station ID for a location and data type"""
    buoy_info = location.get_buoys_for_location(location_name)
    if not buoy_info:
        return None
    return buoy_info.get(data_type)

# Quick test of our functions
test_loc = "lido"
test_time = eastern.localize(datetime.now())
print(f"Testing our data retrieval functions for {test_loc} at {test_time.strftime('%Y-%m-%d %H:%M')}")

buoy_id = get_buoy_id_for_location(test_loc, "swell")
print(f"Swell buoy ID for {test_loc}: {buoy_id}")

swell_data = get_slapp_swell_data(test_loc, test_time)
if swell_data:
    print(f"Successfully retrieved swell data with {len(swell_data[0]['swell_components'])} components")
    # Print the first swell component
    print(f"Primary swell: {swell_data[0]['swell_components']['swell_1']}")
else:
    print("Failed to retrieve swell data")

Testing our data retrieval functions for lido at 2025-05-14 16:38
Swell buoy ID for lido: 44065
Successfully retrieved swell data with 5 components
Primary swell: {'height': 5.16, 'period': 6.25, 'direction': 156.0}
